# Lokalna pretraga

Lokalna pretraga (eng. *local search*, skraćeno LS) tehnika iterativnog poboljšavanja vrednosti jednog rešenja. Na početku algoritma se proizvoljno ili na neki drugi način generiše početno rešenje i izračuna vrednost njegove funkcije cilja. Zatim se vrednost najboljeg rešenja najpre inicijalizuje na vrednost početnog, a potom se algoritam ponavlja kroz nekoliko iteracija. U svakoj iteraciji se razmatra rešenje u okolini trenutnog. Ukoliko je vrednost njegove funkcije cilja bolja od vrednosti funkcije cilja trenutnog rešenja, ažurira se trenutno rešenje. Takođe se, po potrebi, ažurira i vrednost najboljeg dostignutog rešenja. Algoritam se ponavlja dok nije ispunjen kriterijum zaustavljanja. Kriterijum zaustavljanja može biti, na primer, dostignuti maksimalan broj iteracija, dostignuti maksimalan broj ponavljanja najboljeg rešenja, ukupno vreme izvršavanja, itd. 

<img src='assets/local_search.png' style='width:500px'>

Lokalna pretraga se može prikazati sledećim pseudokodom:

* Generisati početno rešenje $s$
* Inicijalizovati vrednost najboljeg rešenja $f^* = f(s)$
* Dok nije ispunjen kriterijum zaustavljanja, ponavljati sledeće korake:
    * Izabrati proizvoljno rešenje $s'$ u okolini rešenja $s$
    * Ako je $f(s') < f(s)$, onda $s = s'$
    * Ako je $f(s') < f^*$, onda $f^* = f(s')$
* Ispisati vrednost rešenja $f^*$

## Primena lokalne pretrage na prost lokacijski problem

Kod prostog lokacijskog problema (eng. *uncapacitated facility location problem*, skraćeno UFLP) poznat je skup korisnika $I$, skup potencijalnih lokacija za resurse $J$, cene dodeljivanja korisnika resursima $c_{ij}$, $i \in I$, $j \in J$ i cene uspostavljanja resursa $f_j$, $j \in J$. Potrebno je odrediti rešenje u kojem je svaki korisnik  pridružen tačno jednom resursu tako da se minimizuje ukupna cena dodeljivanja korisnika resursima i cena upostavljanja resursa. Pritom neki resursi mogu biti neiskorišćeni, a neki iskorišćeni od strane jednog ili više korisnika. 

<img src='assets/UFLP.gif'>

Ovaj problem susrećemo u planiranjima koja treba da rasporede decu po školama ili korisnike nadležnim medicinskim ustanovama. Sam problem je NP težak pa zahteva optimizaciju rešenja.

U ovoj verziji lokalne pretrage se u jednoj iteraciji naredno rešenje bira proizvljno u okolini trenutnog. Inače, mogu se razmatrati sva rešenja iz okoline. Tako se u jednoj verziji algoritma može preći u naredno rešenje čim se dostigne poboljšanje rešenja, a u drugoj se mogu razmotriti sva rešenja i uzeti ono čija je vrednost funkcije cilja najmanja. Ukoliko se radi o maksimizaciji, prihvataju se uvek ona rešenja čija je vrednost funkcije cilja veća. 

Osnovna mana lokalne pretrage se ogleda u tome što najbolje dostignuto lokalno rešenje ne mora ujedno biti i globalno. Ovo se može nadomestiti na razne načine.

U nastavku će biti prikazana implementacija primene lokalne pretrage na UFLP problem.

In [1]:
import numpy as np

In [2]:
np.random.seed(7)

Najpre ćemo napisati funkciju koja učitava ulazne podatke. Pretpostavka je da će se u prvom redu fajla sa podacima naći  broj korisnika `number_of_users` i broj resursa `number_of_resources`, u narednih `number_of_users` redova `number_of_resources` brojeva koji zajedno predstavljaju matricu cena dodeljivanja korisnika resursima koju ćemo zvati `cost`. Niz cena uspostavljanja resursa `fixed_cost` će se učitavati iz poslednjeg reda fajla sa podacima.

In [3]:
!cat data/uflp_input.txt

3 3
1 12 3
2 7 41
19 21 7
12 11 13

In [4]:
def read_input(filename):
    with open(filename, 'r') as input:
        
        number_of_users, number_of_resources = [int(i) for i in input.readline().split()]
        
        cost = [[int(j) for j in input.readline().split()] for i in range(number_of_users)] 
        
        fixed_cost = [int(j) for j in input.readline().split()]
        
    return number_of_users, number_of_resources, cost, fixed_cost

In [5]:
number_of_users, number_of_resources, cost, fixed_cost = read_input('data/uflp_input.txt')

In [6]:
number_of_users

3

In [7]:
number_of_resources

3

In [8]:
cost

[[1, 12, 3], [2, 7, 41], [19, 21, 7]]

In [9]:
fixed_cost

[12, 11, 13]

Funkcijom `is_feasible` se proverava da li je rešenje dopustivo. Rešenje je predstavljeno nizom `solution` koje se sastoji od `number_of_resources` logičkih vrednosti gde vrednost `True` na $j$-tom mestu označava da je resurs uspostavljen, a `False` da nije. Rešenje je dopustivo ukoliko niz `solution` ima bar jednu vrednost `True`.

In [10]:
def is_feasible(number_of_resources, solution):
    for j in range(number_of_resources):
        if solution[j]:
            return True
    return False

Funkcijom `initialize` se generiše proizvoljno početno rešenje. Vrednost `True` će biti izabrana sa verovatnoćom `probability` koja će kasnije biti postavljena na 0.25 budući da se najčešće u optimalnom rešenju nalazi osetno manje uspostavljenih resursa nego neuspostavljenih.

In [11]:
def initialize(number_of_resources, probability):
    solution = []

    # u listu se dodaje True ili False vrednost sa verovatnocom probability
    for j in range(number_of_resources):
        solution.append(np.random.random() < probability)

    # ukoliko u listi nema ni jedne True vrednosti, na proizvoljnu poziciju se upisuje vrednost True
    if not is_feasible(number_of_resources, solution):
        solution[np.random.randint(0, number_of_resources)] = True

    return solution

Funkcijom `restore` se invertuje vrednost $j$-tog indeksa rešenja. Ona će kasnije biti iskorišćena u algoritmu lokalne pretrage za restauiranje vrednosti rešenja ukoliko nije pronađena bolja vrednost.

In [12]:
def restore(j, solution):
    solution[j] = not solution[j]

Funkcijom `solution_value` određuje se vrednost trenutnog rešenja. Za svakog korisnika $i$ se bira resurs $j$ za koji je cena dodeljivanja najmanja. Zatim se taj resurs označava kao zauzet. O zauzetosti resursa se vodi računa u nizu `used` čije su vrednosti na početku postavljene na `False`. Na kraju se i sve vrednosti cena uspostavljanja zauzetih resursa dodaju na povratnu vrednost funkcije. Time je u njoj sadržana ukupna cena dodeljivanja i uspostavljanja za dato fiksno rešenje. Poslednje, niz `solution` se ažurira na osnovu vrednosti niza `used` budući da mogu da postoje u rešenju uspostavljeni resursi koji nisu iskorišćeni.

In [13]:
def solution_value(number_of_users, number_of_resources, cost, fixed_cost, solution):
    
    # ukupna cena
    value = 0.0
    
    # lista pozicija zauzetih resursa
    used = [False] * number_of_resources
    
    
    for i in range(number_of_users):

        # korisnika pridruzujemo najpovoljnijem resursu 
        min_cost = np.inf
        j_used = -1
        for j in range(number_of_resources):
            if solution[j] and cost[i][j] < min_cost:
                min_cost = cost[i][j]
                j_used = j
                
        # potom uvecavamo ukupnu cenu za cenu troskova pridruzivanja
        value += min_cost
        
        # azuriramo listu zauzetih resursa 
        used[j_used] = True
    
    # ukupnu cenu uvecavamo za iznose uspostavljanja resursa
    for j in range(number_of_resources):
        if used[j]:
            value += fixed_cost[j]
            
    # potencijalno azuriramo resenje ukoliko medju inicijalnim resursima ima neiskoriscenih
    solution = used
    
    # vracamo izracunatu ukupnu cenu
    return value

Funkcijom `invert` se bira proizvoljno rešenje iz okoline trenutnog tako što se proizvoljno izabran indeks niza `solution` invertuje čime se uspostavljeni resurs progralašava za neuspostavljeni, ili obratno. Pritom treba voditi računa da se zbog inverzije ne dobije nedopustivo rešenje.

In [14]:
def invert(number_of_resources, solution):
    
    # bira se resenje iz okoline tako sto se na poziciji j invertuje vrednosti
    j = np.random.randint(0, number_of_resources)
    solution[j] = not solution[j]
    
    # ukoliko je ovako dobijeno resenje dopustivo prekidamo pretragu okoline
    if is_feasible(number_of_resources, solution):
        return j
    
    # u suprotnom, ponistavamo promene resenja
    solution[j] = not solution[j]
    
    return -1

Funkcija `local_search` predstavlja samu lokalnu pretragu. Najpre se vrednost najboljeg rešenja postavlja na vrednost tekućeg rešenja. Zatim se u svakoj iteraciji generiše novo rešenje u okolini trenutnog. Ukoliko je vrednost novog rešenja manja od vrednosti trenutnog, ažurira se trenutno rešenje. U suprotnom se restauira njegova prethodna vrednost. U svakoj iteraciji se, po potrebi, ažurira i vrednost najboljeg rešenja.

Kriterijum zaustavljanja će predstavljati dostignut maksimalan broj iteracija.

In [15]:
def local_search(number_of_users, number_of_resources, cost, fixed_cost, solution, max_iters):
    
    # vrednost najboljeg resenja se postavlja na vrednost tekuceg resenja
    curr_value = solution_value(number_of_users, number_of_resources, cost, fixed_cost, solution)
    best_value = curr_value
    best_solution = solution
    
    
    for i in range(0, max_iters):
        
        # generise se novo resenje
        j = invert(number_of_resources, solution)
        if j < 0:
            continue
            
        # izracunava se vrednost novog resenja
        # ukoliko je dobijena vrednost bolja od tekuce vrednosti, generisano resenje predstavlja pocetno resenje sledece iteracije
        # u suprotnom, ponistava se promena resenja i nastavlja se dalja pretraga u odnosu na tekuce resenje 
        new_value = solution_value(number_of_users, number_of_resources, cost, fixed_cost, solution)
        if new_value < curr_value:
            curr_value = new_value
        else:
            restore(j, solution)
            
        # na osnovu dobijenih vrednosti se, po potrebi, azurira i najbolje resenje
        if new_value < best_value:
            best_value = new_value
            best_solution = solution.copy()
        
    return best_value, best_solution

Konačno, nakon učitavanja i inicijalizacije početnog rešenja, vrednost rešenja je povratna vrednost lokalne pretrage.

In [16]:
solution = initialize(number_of_resources, 0.25)
best_value, best_solution = local_search(number_of_users, number_of_resources, cost, fixed_cost, solution, 10000)

In [17]:
print('Najmanja cena:', best_value)

Najmanja cena: 34.0


In [18]:
print('Optimalno resenje: ', best_solution)

Optimalno resenje:  [True, False, False]


Optimalno rešenje se dobija uspostavljanjem samo prvog resursa i dodeljivanjem svih korisnika njemu. Cene dodeljivanja prvom resursu su redom 1, 2 i 19, a cena uspostavljanja 12. Dakle, vrednost optimalnog rešenja iznosi $1+2+19+12=34$.

Algoritam lokalne pretrage za UFLP se može ubrzati imajući u vidu činjenicu da u svakoj iteraciji najviše vremena oduzima računanje funkcije cilja, a da se u svakom koraku, pri prelasku u naredno rešenje, može ažurirati njena trenutna vrednost, umesto ponovnog računanja. Ako se pri prelasku na naredno rešenje vrši uspostavljanje $j$-tog resursa dovoljno je proveriti za svakog korisnika $i$ da li je vrednost $c_{ij}$ manja od vrednosti $c_{i,best(i)}$, gde je $best(i)$ indeks resursa kome je korisnik $i$ dodeljen. Ukoliko ova vrednost jeste manja, potrebno
je ažurirati vrednost funkcije cilja. Složenost ovog koraka je linearna za razliku od kvadratnog vremena koje treba utrošiti za ponovno računanje funkcije cilja. Ako se pri prelasku u naredno rešenje vrši oslobađanje $j$-tog resursa, potrebno je, umesto svih korisnika iz skupa $I$, razmotriti samo one koji su dodeljeni resursu $j$. Kako je u prosečnom slučaju broj korisnika dodeljenih jednom resursu znatno manji od ukupnog broja resursa, ovo znatno ubrzava vreme pretrage u poređenju sa vremenom koje bi se potrošilo za ponovno računanje vrednosti funkcije cilja.